In [ ]:
#! pip install transformers datasets evaluate

In [121]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    print("GPU: ", torch.cuda.get_device_name(0))
else:
    print("CPU")

CPU


In [122]:
import pandas as pd
from datasets import load_dataset

In [123]:
dataset_train = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[:1%]")

dataset_eval = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[1%:2%]")

Using custom data configuration default-95c8ce64d75a95bb
Found cached dataset csv (/home/jawad/.cache/huggingface/datasets/csv/default-95c8ce64d75a95bb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-95c8ce64d75a95bb
Found cached dataset csv (/home/jawad/.cache/huggingface/datasets/csv/default-95c8ce64d75a95bb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [124]:
dataset_train = dataset_train.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

dataset_eval = dataset_eval.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

## Tokenizer

In [125]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

loading configuration file config.json from cache at /home/jawad/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /home/jawad/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
loading f

In [128]:
# apply tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], 
                     padding="max_length", 
                     truncation=True, 
                     max_length=100)


tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_eval = dataset_eval.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# https://download.pytorch.org/whl/cu116
# !pip3 install torch torchvision torchaudio --extra-index-url 

## Process labels

In [129]:
def minus_1(example):
    example["label"] = example["label"] - 1
    return example

In [130]:
tokenized_dataset_train = tokenized_dataset_train.map(minus_1)
tokenized_dataset_eval = tokenized_dataset_eval.map(minus_1)

  0%|          | 0/1000 [00:00<?, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [133]:
tokenized_dataset_train.to_pandas()
# how to reduce padding ?

,label,text,input_ids,token_type_ids,attention_mask
0,4,I have bought several of the Vitality canned d...,"[101, 146, 1138, 3306, 1317, 1104, 1103, 25118...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,0,Product arrived labeled as Jumbo Salted Peanut...,"[101, 22249, 2474, 12893, 1112, 23915, 13685, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,3,This is a confection that has been around a fe...,"[101, 1188, 1110, 170, 14255, 11916, 1988, 111...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,If you are looking for the secret ingredient i...,"[101, 1409, 1128, 1132, 1702, 1111, 1103, 3318...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,4,Great taffy at a great price. There was a wid...,"[101, 2038, 27629, 12383, 1120, 170, 1632, 394...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
...,...,...,...,...,...
995,4,BLACK MARKET HOT SAUCE IS WONDERFUL.... My hus...,"[101, 139, 10783, 1658, 2428, 9960, 2069, 2244...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
996,4,"Man what can i say, this salsa is the bomb!! i...","[101, 2268, 1184, 1169, 178, 1474, 117, 1142, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
997,4,this sauce is so good with just about anything...,"[101, 1142, 14313, 1110, 1177, 1363, 1114, 119...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
998,0,Not hot at all. Like the other low star review...,"[101, 1753, 2633, 1120, 1155, 119, 2409, 1103,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Load the model

Start by loading your model and specify the number of expected labels.

In [136]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", 
                                                           num_labels=5)

loading configuration file config.json from cache at /home/jawad/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "typ

[Training hyperparameters](https://huggingface.co/docs/transformers/v4.28.1/en/main_classes/trainer#transformers.TrainingArguments) - Specify where to save the checkpoints from your training.
specify the evaluation_strategy parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [137]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", 
                                  optim="adamw_torch", 
                                  save_strategy='epoch',
                                  evaluation_strategy="epoch"
                                 )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## Metric

In [138]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [139]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Trainer

In [140]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    compute_metrics=compute_metrics,
)

In [141]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 375
  Number of trainable parameters = 108314117


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.953758,0.635000
2,No log,0.819800,0.700000
3,No log,0.852521,0.707000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-125
Configuration saved in test_trainer/checkpoint-125/config.json
Model weights saved in test_trainer/checkpoint-125/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-250
Configuration saved in test_trainer/checkpoint-250/config.json
Model weights saved in test_trainer/che

TrainOutput(global_step=375, training_loss=0.812508544921875, metrics={'train_runtime': 1881.2451, 'train_samples_per_second': 1.595, 'train_steps_per_second': 0.199, 'total_flos': 154170786600000.0, 'train_loss': 0.812508544921875, 'epoch': 3.0})

## Evaluation

using whole eval dataset

In [142]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8


{'eval_loss': 0.8525205254554749,
 'eval_accuracy': 0.707,
 'eval_runtime': 93.0332,
 'eval_samples_per_second': 10.749,
 'eval_steps_per_second': 1.344,
 'epoch': 3.0}

Using validation dataset

In [143]:
dataset_validation = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[30%:31%]")

Using custom data configuration default-95c8ce64d75a95bb
Found cached dataset csv (/home/jawad/.cache/huggingface/datasets/csv/default-95c8ce64d75a95bb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [144]:
dataset_validation = dataset_validation.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

In [145]:
tokenized_dataset_validation = dataset_validation.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [146]:
tokenized_dataset_validation = tokenized_dataset_validation.map(minus_1)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [147]:
predictions = trainer.predict(tokenized_dataset_validation)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


In [148]:
predictions.metrics

{'test_loss': 0.9764316082000732,
 'test_accuracy': 0.671,
 'test_runtime': 109.2841,
 'test_samples_per_second': 9.15,
 'test_steps_per_second': 1.144}

## Prediction

In [149]:
dataset_new = load_dataset("csv", data_files="amazon Food Reviews 100k Dataset.csv", 
                       split="train[3%:4%]")

Using custom data configuration default-95c8ce64d75a95bb
Found cached dataset csv (/home/jawad/.cache/huggingface/datasets/csv/default-95c8ce64d75a95bb/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [150]:
dataset_new = dataset_new.rename_column("Review", "text").remove_columns(['Id', 'Rating'])

In [151]:
tokenized_dataset_new = dataset_new.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [152]:
logits_new = trainer.predict(tokenized_dataset_new)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


In [153]:
predictions = np.argmax(logits_new.predictions, axis=-1)

In [154]:
predictions[:4]

array([4, 4, 4, 4])

## Prediction using Pipeline

In [111]:
label_names = [ 'Very Bad', 'Bad', 'Neutral', 'Good', 'Very Good']

In [118]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

# Load the tokenizer and model
#model = AutoModelForSequenceClassification.from_pretrained("test_trainer/checkpoint-125/")
config = AutoConfig.from_pretrained("test_trainer/checkpoint-125/",
                                    num_labels=len(label_names), 
                                    id2label={i: name for i, name in enumerate(label_names)},
                                    label2id={name: i for i, name in enumerate(label_names)})
# Load the model with the updated configuration
model = AutoModelForSequenceClassification.from_pretrained("test_trainer/checkpoint-125/", config=config)


loading configuration file test_trainer/checkpoint-125/config.json
Model config BertConfig {
  "_name_or_path": "test_trainer/checkpoint-125/",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Very Bad",
    "1": "Bad",
    "2": "Neutral",
    "3": "Good",
    "4": "Very Good"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "Bad": 1,
    "Good": 3,
    "Neutral": 2,
    "Very Bad": 0,
    "Very Good": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size

In [119]:
# Define the pipeline
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer
)

# Use the pipeline to predict the sentiment of a sentence
result = classifier("I loved the movie!")
print(result)

[{'label': 'Very Good', 'score': 0.8186774253845215}]
